In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('master_books.csv')
print(df.shape)
df.head()

(19707, 6)


,title,author,rating,description,language,isbn
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"4.33 avg rating — 6,191,931 ratings",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,0439023483 (ISBN13: 9780439023481)
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,"4.50 avg rating — 2,414,060 ratings",There is a door at the end of a silent corrido...,English,0439358078 (ISBN13: 9780439358071)
2,To Kill a Mockingbird,Harper Lee,"4.28 avg rating — 4,348,656 ratings",The unforgettable novel of a childhood in a sl...,English,NaN
3,Pride and Prejudice,Jane Austen,"4.26 avg rating — 2,886,703 ratings",Alternate cover edition of ISBN 9780679783268S...,English,NaN
4,"Twilight (Twilight, #1)",Stephenie Meyer,"3.59 avg rating — 4,815,701 ratings",About three things I was absolutely positive.\...,English,0316015849 (ISBN13: 9780316015844)


In [8]:
df['language'].value_counts()

English                                15960
Arabic                                   441
Spanish                                  278
French                                   220
German                                   160
Indonesian                               159
Portuguese                               114
Italian                                   90
Dutch                                     86
Turkish                                   77
Persian                                   54
Polish                                    41
Romanian                                  37
Malay                                     33
Czech                                     33
Bulgarian                                 31
Swedish                                   25
Japanese                                  24
Greek, Modern (1453-)                     21
Urdu                                      21
Russian                                   18
Bengali                                   17
Serbian   

In [9]:
df = df[df['language'] == 'English']
df.shape

(15960, 6)

In [10]:
df.isnull().sum()

title             0
author            0
rating            0
description     148
language          0
isbn           2754
dtype: int64

In [11]:
import spacy

In [12]:
nlp = spacy.load('en_core_web_sm')

In [16]:
text = df.loc[:5, 'description']
text = ''.join(text).replace('\r', '').replace('\n', '')
text

'WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister\'s place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming 

In [17]:
doc = nlp(text)

In [18]:
[token.lemma_ for token in doc]

['WINNING',
 'mean',
 'FAME',
 'and',
 'FORTUNE.LOSING',
 'mean',
 'CERTAIN',
 'death.the',
 'HUNGER',
 'GAMES',
 'have',
 'BEGUN',
 '.',
 '.',
 '.',
 '.In',
 'the',
 'ruin',
 'of',
 'a',
 'place',
 'once',
 'know',
 'as',
 'North',
 'America',
 'lie',
 'the',
 'nation',
 'of',
 'Panem',
 ',',
 'a',
 'shine',
 'Capitol',
 'surround',
 'by',
 'twelve',
 'outlying',
 'district',
 '.',
 'the',
 'Capitol',
 'be',
 'harsh',
 'and',
 'cruel',
 'and',
 'keep',
 'the',
 'district',
 'in',
 'line',
 'by',
 'force',
 '-PRON-',
 'all',
 'to',
 'send',
 'one',
 'boy',
 'and',
 'once',
 'girl',
 'between',
 'the',
 'age',
 'of',
 'twelve',
 'and',
 'eighteen',
 'to',
 'participate',
 'in',
 'the',
 'annual',
 'Hunger',
 'Games',
 ',',
 'a',
 'fight',
 'to',
 'the',
 'death',
 'on',
 'live',
 'tv.sixteen',
 '-',
 'year',
 '-',
 'old',
 'Katniss',
 'Everdeen',
 'regard',
 '-PRON-',
 'as',
 'a',
 'death',
 'sentence',
 'when',
 '-PRON-',
 'step',
 'forward',
 'to',
 'take',
 '-PRON-',
 'sister',
 "'s"